In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import sys
sys.path.append('/home/rustam/hyperspecter_segmentation/makitorch')
sys.path.append('/home/rustam/hyperspecter_segmentation/')
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter
import cv2
from hsi_dataset_api import HsiDataset
from makitorch.dataloaders.HsiDataloader import HsiDataloader

from makitorch.data_tools.augmentation import DataAugmentator
from makitorch.data_tools.augmentation import BaseDataAugmentor
from makitorch.data_tools.preprocessing import BaseDataPreprocessor
from makitorch.data_tools.preprocessing import DataPreprocessor
from makitorch.dataset_remapper import DatasetRemapper

from typing import Callable, Optional, Union
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms as T
import torchvision.transforms.functional as TF
from torchvision import utils
import cv2
from Losses import FocalLoss
import matplotlib.pyplot as plt

import random
from multiprocessing.dummy import Pool
from multiprocessing import shared_memory
import seaborn as sns
import json
import glob
from tqdm import tqdm


DATA_SHAPE = (128, 128)
INPUT_DIM = 17
NUM_CLASSES = None
PATH_DATA = '/raid/rustam/hyperspectral_dataset/new_cropped_hsi_data'

device = 'cuda:0'

PATH_TO_SAVE = '/raid/rustam/hyperspectral_dataset/diff_exp_with_other__attention_with_other/result_masks_0'

ATT_EXP = 'AttentionV11'
WO_ATT_EXP = 'AttentionV11-WO'
SW_ATT_EXP = 'SW-AttentionV11'
SW_WO_ATT_EXP = 'SW-AttentionV11-WO'


id2exp_name = {
    '949923586': 'AttentionV11 kfold0 R1',
    '121120867': 'AttentionV11 kfold0 R2',
    '847352398': 'AttentionV11 kfold0 R3',
    '119260030': 'AttentionV11 kfold0 R4',
    '618375711': 'AttentionV11-WO kfold0 R1',
    '898124683': 'AttentionV11-WO kfold0 R2',
    '109979765': 'AttentionV11-WO kfold0 R3',
    '633351465': 'AttentionV11-WO kfold0 R4',
    '795994918': 'SW-AttentionV11 kfold0 R1',
    '596305665': 'SW-AttentionV11 kfold0 R2',
    '344737891': 'SW-AttentionV11 kfold0 R3',
    '85837259':  'SW-AttentionV11 kfold0 R4',
    '782108417': 'SW-AttentionV11-WO kfold0 R1',
    '783196294': 'SW-AttentionV11-WO kfold0 R2',
    '479759236': 'SW-AttentionV11-WO kfold0 R3',
    '955604717': 'SW-AttentionV11-WO kfold0 R4',

    '279928761': 'AttentionV11 kfold1 R1',
    '996122000': 'AttentionV11 kfold1 R2',
    '312177183': 'AttentionV11 kfold1 R3',
    '663225523': 'AttentionV11 kfold1 R4',
    '930756249': 'AttentionV11-WO kfold1 R1',
    '946444593': 'AttentionV11-WO kfold1 R2',
    '527159228': 'AttentionV11-WO kfold1 R3',
    '27327763':  'AttentionV11-WO kfold1 R4',
    '373915631': 'SW-AttentionV11 kfold1 R1',
    '812373490': 'SW-AttentionV11 kfold1 R2',
    '927094798': 'SW-AttentionV11 kfold1 R3',
    '784767717': 'SW-AttentionV11 kfold1 R4',
    '829169':    'SW-AttentionV11-WO kfold1 R1',
    '970298918': 'SW-AttentionV11-WO kfold1 R2',
    '24035521':  'SW-AttentionV11-WO kfold1 R3',
    '862357190': 'SW-AttentionV11-WO kfold1 R4',

    '665848744': 'AttentionV11 kfold2 R1',
    '527008414': 'AttentionV11 kfold2 R2',
    '962896694': 'AttentionV11 kfold2 R3',
    '257257676': 'AttentionV11 kfold2 R4',
    '314015568': 'AttentionV11-WO kfold2 R1',
    '410030650': 'AttentionV11-WO kfold2 R2',
    '942004738': 'AttentionV11-WO kfold2 R3',
    '437994589': 'AttentionV11-WO kfold2 R4',
    '747074509': 'SW-AttentionV11 kfold2 R1',
    '983492227': 'SW-AttentionV11 kfold2 R2',
    '238334469': 'SW-AttentionV11 kfold2 R3',
    '826242444': 'SW-AttentionV11 kfold2 R4',
    '443269504': 'SW-AttentionV11-WO kfold2 R1',
    '259831776': 'SW-AttentionV11-WO kfold2 R2',
    '347933396': 'SW-AttentionV11-WO kfold2 R3',
    '241122960': 'SW-AttentionV11-WO kfold2 R4',
}

exp_name2id = dict([(v, k) for k, v in id2exp_name.items()])

In [ ]:
#
# ----------------------------------------------------------------
#                           MODEL
# ----------------------------------------------------------------
#
# Paper must read:
# https://arxiv.org/pdf/1904.11492.pdf
#
# Original paper:
# https://arxiv.org/pdf/1906.02849.pdf
#
# Github:
# https://github.com/sinAshish/Multi-Scale-Attention
#


class PAM_Module(nn.Module):
    """ 
    Position attention module

    """
    #Ref from SAGAN
    def __init__(self, in_dim, dim_reduse: int = 8):
        super(PAM_Module, self).__init__()
        self.chanel_in = in_dim

        self.query_conv = nn.Conv2d(
            in_channels=in_dim, out_channels=in_dim // dim_reduse, 
            kernel_size=1
        )
        self.key_conv = nn.Conv2d(
            in_channels=in_dim, out_channels=in_dim // dim_reduse,
             kernel_size=1
        )
        self.value_conv = nn.Conv2d(
            in_channels=in_dim, out_channels=in_dim, 
            kernel_size=1
        )
        self.gamma = nn.Parameter(torch.zeros(1))
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, return_attention=False):
        """
        Parameters:
        ----------
            inputs :
                x : input feature maps( B X C X H X W)
            returns :
                out : attention value + input feature
                attention: B X (HxW) X (HxW)
        """
        m_batchsize, C, height, width = x.size()
        proj_query = self.query_conv(x).view(m_batchsize, -1, width*height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(m_batchsize, -1, width*height)

        energy = torch.bmm(proj_query, proj_key)
        attention = self.softmax(energy)
        proj_value = self.value_conv(x).view(m_batchsize, -1, width*height)

        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(m_batchsize, C, height, width)

        out = self.gamma * out

        if return_attention:
            return out, attention

        return out


class CAM_Module(nn.Module):
    """ 
    Channel attention module

    """
    def __init__(self, in_dim):
        super(CAM_Module, self).__init__()
        self.chanel_in = in_dim
        
        self.gamma = nn.Parameter(torch.zeros(1))
        self.softmax  = nn.Softmax(dim=-1)

    def forward(self, x, return_attention=False):
        """
        Parameters:
        ----------
            inputs :
                x : input feature maps( B X C X H X W)
            returns :
                out : attention value + input feature
                attention: B X C X C
        """
        m_batchsize, C, height, width = x.size()
        proj_query = x.view(m_batchsize, C, -1)
        proj_key = x.view(m_batchsize, C, -1).permute(0, 2, 1)
       
        energy = torch.bmm(proj_query, proj_key)
        energy_new = torch.max(energy, -1, keepdim=True)[0].expand_as(energy)-energy # swap?
        attention = self.softmax(energy_new) # Sigmoid?
        proj_value = x.view(m_batchsize, C, -1)

        out = torch.bmm(attention, proj_value)
        out = out.view(m_batchsize, C, height, width)

        out = self.gamma * out

        if return_attention:
            return out, attention

        return out



class MySuperAttentionNetLittleInput(nn.Module):
    
    def __init__(self, in_f=237, out_f=NUM_CLASSES, *args):
        super().__init__()
        #self.bn_start = nn.BatchNorm3d(in_f)

        self.backbone = nn.Sequential(
            # (N, in_f, 128, 128)
            nn.Conv2d(in_f, 64, kernel_size=5, stride=2, padding=2, bias=False),
            # (N, 128, 64, 64)
            nn.BatchNorm2d(64),
            nn.ReLU(),
        
            nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2, bias=False),
            # (N, 128, 32, 32)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False),
            # (N, 128, 32, 32)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False),
            # (N, 128, 32, 32)
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )
        # Output (N, 128, 32, 32)
        self.cam_module = CAM_Module(128)
        # Output (N, 128, 32, 32)
        self.pam_module = PAM_Module(128, dim_reduse=8)
        # Concat out_f and out_f = out_f * 2
        self.final_backbone = nn.Sequential(
            nn.Conv2d(128 * 3, 128, kernel_size=1, stride=1, padding=0, bias=False),
            # (N, 128, 32, 32)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False),
            # (N, 128, 32, 32)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.ConvTranspose2d(128, 128, kernel_size=5, stride=2, padding=2, output_padding=1, bias=False),
            # (N, 128, 64, 64)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2, padding=2, output_padding=1, bias=False),
            # (N, 64, 128, 128)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.Conv2d(64, out_f, kernel_size=5, stride=1, padding=2, bias=False),
            # (N, out_f, 128, 128)
            nn.BatchNorm2d(out_f),
            nn.ReLU(),
            # Final conv
            nn.Conv2d(out_f, out_f, kernel_size=1, stride=1, padding=0, bias=False),
        )
    
    def forward(self, x):
        x = self.backbone(x)

        cam_x = self.cam_module(x)
        pam_x = self.pam_module(x)

        x = torch.cat([cam_x, pam_x, x], dim=1)
        x = self.final_backbone(x)

        return x

     
class MySuperNetLittleInput(nn.Module):
    
    def __init__(self, in_f=237, out_f=NUM_CLASSES, *args):
        super().__init__()
        #self.bn_start = nn.BatchNorm3d(in_f)

        self.backbone = nn.Sequential(
            # (N, in_f, 128, 128)
            nn.Conv2d(in_f, 64, kernel_size=5, stride=2, padding=2, bias=False),
            # (N, 128, 64, 64)
            nn.BatchNorm2d(64),
            nn.ReLU(),
        
            nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2, bias=False),
            # (N, 128, 32, 32)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False),
            # (N, 128, 32, 32)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 128 * 3, kernel_size=3, stride=1, padding=1, bias=False),
            # (N, 128 * 3, 32, 32)
            nn.BatchNorm2d(128 * 3),
            nn.ReLU(),
        )
        self.final_backbone = nn.Sequential(
            nn.Conv2d(128 * 3, 128, kernel_size=1, stride=1, padding=0, bias=False),
            # (N, 128, 32, 32)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False),
            # (N, 128, 32, 32)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.ConvTranspose2d(128, 128, kernel_size=5, stride=2, padding=2, output_padding=1, bias=False),
            # (N, 128, 64, 64)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2, padding=2, output_padding=1, bias=False),
            # (N, 64, 128, 128)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.Conv2d(64, out_f, kernel_size=5, stride=1, padding=2, bias=False),
            # (N, out_f, 128, 128)
            nn.BatchNorm2d(out_f),
            nn.ReLU(),
            # Final conv
            nn.Conv2d(out_f, out_f, kernel_size=1, stride=1, padding=0, bias=False),
        )
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.final_backbone(x)
        return x


In [ ]:
class ShmData:

    def __init__(self, shm_name, shape, dtype):
        self.shm_name = shm_name
        self.shape = shape
        self.dtype = dtype

class HsiDataloaderCutter(torch.utils.data.IterableDataset):
    def __init__(
            self, 
            images, masks,
            augmentation: Optional[Union[DataAugmentator, Callable]] = BaseDataAugmentor(),
            shuffle_data=False,
            cut_window=(8, 8),
            map_mask_to_class=False,
            shared_memory_imgs_data: ShmData = None,
            shared_memory_masks_data: ShmData = None
        ):
        super().__init__()
        self.shuffle_data = shuffle_data
        self.augmentation = augmentation
        if cut_window is not None and cut_window[0] == 1 and cut_window[1] == 1:
            self.ignore_image_augs = True
        else:
            self.ignore_image_augs = False
        self.cut_window = cut_window
        self.map_mask_to_class = map_mask_to_class
        self.shared_memory_imgs_data = shared_memory_imgs_data
        self.shared_memory_masks_data = shared_memory_masks_data
        
        self.shm_imgs: shared_memory.SharedMemory = None
        self.shm_masks: shared_memory.SharedMemory = None

        self.images = images
        self.masks = masks

    def __iter__(self):
        assert self.images is not None and self.masks is not None
        if self.shuffle_data:
            self.images, self.masks = shuffle(self.images, self.masks)
        
        for image, mask in zip(self.images, self.masks):
            yield self.augmentation(
                image, mask, 
                map_mask_to_class=self.map_mask_to_class,
                ignore_image_augs=self.ignore_image_augs
            )


def cut_into_parts_model_input(
        image: np.ndarray, h_parts: int, 
        w_parts: int, h_win: int, w_win: int):
    image_parts_list = []

    for h_i in range(h_parts):
        for w_i in range(w_parts):
            img_part = image[:, :,  
                h_i * h_win: (h_i+1) * h_win, 
                w_i * w_win: (w_i+1) * w_win
            ]

            image_parts_list.append(img_part)
    return image_parts_list


def merge_parts_into_single_mask(
        preds, shape, h_parts: int, 
        w_parts: int, h_win: int, w_win: int):
    pred_mask = torch.zeros(
        shape,
        dtype=preds.dtype, device=preds.device
    )
    counter = 0

    for h_i in range(h_parts):
        for w_i in range(w_parts):
            pred_mask[:, :,  
                h_i * h_win: (h_i+1) * h_win, 
                w_i * w_win: (w_i+1) * w_win
            ] = preds[counter]
            counter += 1
    return pred_mask


def collect_prediction_and_target(
        eval_loader, model, cut_window=(8, 8), 
        image_shape=(512, 512), num_classes=NUM_CLASSES,
        divided_batch=2):
    target_list = []
    pred_list = []
    
    for in_data_x, val_data in iter(eval_loader):
        batch_size = in_data_x.shape[0]
        # We will cut image into peases and stack it into single BIG batch
        h_win, w_win = cut_window
        h_parts, w_parts = image_shape[1] // w_win, image_shape[0] // h_win
        in_data_x_parts_list = cut_into_parts_model_input(
            in_data_x, h_parts=h_parts, 
            w_parts=w_parts, h_win=h_win, w_win=w_win
        )
        in_data_x_batch = torch.cat(in_data_x_parts_list, dim=0) # (N, 17, 1, 1)
        # Make predictions
        part_divided = len(in_data_x_batch) // divided_batch
        pred_batch_list = []
        for b_i in range(part_divided):
            if b_i == (part_divided - 1):
                # last
                single_batch = in_data_x_batch[b_i * divided_batch:]
            else:
                single_batch = in_data_x_batch[b_i * divided_batch: (b_i+1) * divided_batch]
            # Make predictions
            preds = model(single_batch) # (divided_batch, num_classes)
            pred_batch_list.append(preds)
        preds = torch.cat(pred_batch_list, dim=0) 
        # Create full image again from peases
        pred_mask = merge_parts_into_single_mask(
            preds=preds, shape=(batch_size, num_classes, image_shape[0], image_shape[1]), 
            h_parts=h_parts, w_parts=w_parts, h_win=h_win, w_win=w_win
        )
        target_list.append(val_data)
        pred_list.append(pred_mask)
    return (torch.cat(pred_list, dim=0), 
            torch.cat(target_list, dim=0)
    )
        

def calculate_iou(pred_list, target_list, num_classes=NUM_CLASSES):
    res_list = []
    
    for preds, target in zip(pred_list, target_list):
        # preds - (num_classes, H, W)
        preds = preds.detach()
        # target - (H, W)
        target = target.detach()

        preds = nn.functional.softmax(preds, dim=0)
        preds = torch.argmax(preds, dim=0)
        
        preds_one_hoted = torch.nn.functional.one_hot(preds, num_classes).view(-1, num_classes).cpu()
        target_one_hoted = torch.nn.functional.one_hot(target, num_classes).view(-1, num_classes).cpu()
        res = jaccard_score(target_one_hoted, preds_one_hoted, average=None, zero_division=1)
        res_list.append(
            res
        )
    
    res_np = np.stack(res_list)
    #res_np = res_np.mean(axis=0)
    return res_np


def dice_loss(preds, ground_truth, eps=1e-5, dim=None, use_softmax=False, softmax_dim=1):
    """
    Computes Dice loss according to the formula from:
    V-Net: Fully Convolutional Neural Networks forVolumetric Medical Image Segmentation
    Link to the paper: http://campar.in.tum.de/pub/milletari2016Vnet/milletari2016Vnet.pdf
    Parameters
    ----------
    preds : tf.Tensor
        Predicted probabilities.
    ground_truth : tf.Tensor
        Ground truth labels.
    eps : float
        Used to prevent division by zero in the Dice denominator.
    axes : list
        Defines which axes the dice value will be computed on. The computed dice values will be averaged
        along the remaining axes. If None, Dice is computed on an entire batch.
    Returns
    -------
    tf.Tensor
        Scalar dice loss tensor.
    """
    ground_truth = ground_truth.float().to(device=preds.device)
    
    if use_softmax:
        preds = nn.functional.softmax(preds, dim=softmax_dim)
    
    numerator = preds * ground_truth
    numerator = torch.sum(numerator, dim=dim)

    p_squared = torch.square(preds)
    p_squared = torch.sum(p_squared, dim=dim)
    # ground_truth is not squared to avoid unnecessary computation.
    # 0^2 = 0
    # 1^2 = 1
    g_squared = torch.sum(torch.square(ground_truth), dim=dim)
    denominator = p_squared + g_squared + eps

    dice = 2 * numerator / denominator
    return 1 - dice


def take_pred_masks(preds):
    preds = nn.functional.softmax(preds, dim=1)
    preds = torch.argmax(preds, dim=1)
    return preds



def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if getattr(m, 'bias') is not None:
            m.bias.data.fill_(0.01)

In [ ]:
#
# ----------------------------------------------------------------
#                  PYTORCH LIGHTNING MODEL
# ----------------------------------------------------------------
#

class NnModel(pl.LightningModule):
    def __init__(
            self, model, loss,
            T_0=10, T_mult=2, experiment=None, enable_image_logging=True,
            lr=1e-3, lr_list=None, epoch_list=None):
        super().__init__()
        self.model = model
        self.loss = loss
        self.lr = lr
        self.experiment = experiment
        self.enable_image_logging = enable_image_logging

        if lr_list is not None and epoch_list is not None:
            if len(lr_list) != len(epoch_list):
                raise ValueError(
                    f"lr_list={lr_list} and epoch_list={epoch_list}"+\
                    " must be arrays of same length"
                )
            print(
                f'Using dynamic lr with next setup: lr_list={lr_list} epoch_list={sorted(epoch_list)}.\n+'+\
                'Epoch list is sorted by increasing.'
            )
            # Further lr/epoch value will be added/deleted into list
            # Make copy to make sure that original data is safe
            lr_list = copy.deepcopy(lr_list)
            epoch_list = copy.deepcopy(epoch_list)

            self.lr_list = lr_list
            self.epoch_list = sorted(epoch_list)
            self.is_lr_must_change = True
        else:
            self.is_lr_must_change = False

        self.T_0 = T_0
        self.T_mult = T_mult
            
    def forward(self, x):
        out = self.model(x)
        return out
    
    def on_train_epoch_start(self):
        # Stuff here - for change learning rate on a fly
        if self.is_lr_must_change and len(self.epoch_list) != 0:
            # Check, current epoch bigger than epoch on which must be updated lr
            if self.current_epoch >= self.epoch_list[0]:
                # Update optimizer
                self.lr = self.lr_list[0]
                self.trainer.accelerator.setup_optimizers(self.trainer)
                # Clear used variables
                del self.epoch_list[0]
                del self.lr_list[0]

    def configure_optimizers(self):
        # Change lr after some epoch, original idea from:
        # https://github.com/PyTorchLightning/pytorch-lightning/issues/3095
        # But here I do something different and its more what I like
        print(f'Init optimizer with params: lr={self.lr}, T_0={self.T_0}, T_mult={self.T_mult}')
        optimizer = optim.Adam( 
            self.parameters(), lr=self.lr
        )
        lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, 
            T_0=self.T_0, T_mult=self.T_mult, eta_min=0
        )
        return { "optimizer": optimizer, "lr_scheduler": lr_scheduler }
    
    def training_step(self, train_batch, batch_idx):
        img, mask = train_batch
        preds = self.model(img) # (N, C, H, W)
        loss = self.loss(preds, mask) # (N, H, W)
        self.log('train_loss', loss)
        if self.experiment is not None:
            self.experiment.log_metric("train_loss", loss, epoch=self.current_epoch, step=self.global_step)
        return loss

    def validation_step(self, batch, batch_idx):
        return batch
    
    def validation_epoch_end(self, outputs):
        print('VALIDATION: Size epoch end input with data n=', len(outputs))
        
        def create_big_image(tensor, single_elem_shape):
            shape = tensor.shape
            tensor = torch.cat([
                t_s.view(*single_elem_shape)
                for t_s in tensor
            ], dim=-1)
            return tensor
        # (batch_size, NUM_CLASSES, H, W) | (batch_size, H, W)
        pred_tensor, target_tensor = collect_prediction_and_target(outputs, self.model)
        pred_as_mask = take_pred_masks(pred_tensor) # (batch_size, H, W)
        pred_big_tensor = create_big_image(
            pred_tensor, 
            single_elem_shape=[1, NUM_CLASSES, DATA_SHAPE[0], DATA_SHAPE[1]]
        )
        target_big_tensor = create_big_image(
            target_tensor, 
            single_elem_shape=[1, DATA_SHAPE[0], DATA_SHAPE[1]]
        )

        target_one_hotted_tensor = torch.nn.functional.one_hot(
            target_big_tensor, NUM_CLASSES # Num classes
        )
        # (N, H, W, C) --> (N, C, H, W)
        target_one_hotted_tensor = target_one_hotted_tensor.permute(0, -1, 1, 2)
        dice_loss_val = dice_loss(
            pred_big_tensor, target_one_hotted_tensor, 
            dim=[0, 2, 3], use_softmax=True, softmax_dim=1,
        )
        metric = calculate_iou(pred_big_tensor, target_big_tensor)
        
        if self.enable_image_logging:
            for batch_indx, (target_s, pred_s) in enumerate(zip(target_tensor, pred_as_mask)):
                target_s = target_s.squeeze()
                pred_s = pred_s.squeeze()
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
                sns.heatmap(pred_s.cpu().detach().numpy(), ax=ax1, vmin=0, vmax=NUM_CLASSES)
                sns.heatmap(target_s.cpu().detach().numpy(), ax=ax2, vmin=0, vmax=NUM_CLASSES)
                fig.savefig(f'temp_fig_{GPU_ID}.png')
                plt.close(fig)

                if self.experiment is not None and self.current_epoch != 0:
                    self.experiment.log_image(
                        f'temp_fig_{GPU_ID}.png', name=f'{batch_indx}', 
                        overwrite=False, step=self.global_step
                    )
                # Sometimes - send too much images to comet - slow training
                # So, for training time skip some samples - its better to do this after 
                # training on the best model
                if batch_indx == NUMBER_RESULTS_TO_PLOT:
                    break

        if self.experiment is not None and self.current_epoch != 0:
            self.experiment.log_confusion_matrix(
                target_tensor.cpu().detach().numpy().reshape(-1), 
                torch.stack(
                    [elem.cpu() for elem in pred_as_mask], 
                    dim=0
                ).cpu().detach().numpy().reshape(-1)
            )

        mean_dice_loss_per_class_dict = {
            f"mean_dice_loss_per_class_{i}": d_l.float()
            for i, d_l in enumerate(dice_loss_val)
        }
        mean_dice_loss_dict = {
            f"mean_dice_loss": dice_loss_val.mean().float()
        }
        mean_iou_class_dict = {
            f"mean_iou_class_{i}": torch.tensor(iou, dtype=torch.float)
            for i, iou in enumerate(metric.mean(axis=0))
        }
        mean_iou_dict = {
            "mean_iou": float(metric.mean()),
        }
        
        # Log will save stuff in comet
        self.log_dict(mean_iou_dict)
        
        if self.experiment is not None and self.current_epoch != 0:
        
            self.experiment.log_metrics(
                mean_dice_loss_per_class_dict,
                epoch=self.current_epoch
            )

            self.experiment.log_metrics(
                mean_dice_loss_dict,
                epoch=self.current_epoch
            )

            self.experiment.log_metrics(
                mean_iou_class_dict,
                epoch=self.current_epoch
            )

            self.experiment.log_metrics(
                mean_iou_dict,
                epoch=self.current_epoch
            )
        else:
            print(mean_dice_loss_per_class_dict)
            print(mean_dice_loss_dict)
            print(mean_iou_class_dict)
            print(mean_iou_dict)
            print('---------------------------------')

    def test_step(self, batch, batch_idx):
        return batch

    def test_epoch_end(self, outputs):
        # This function almost same as validation_epoch_end - but its only for test data
        # TODO: Make it as one function?
        print('TEST: Performe test with data n=', len(outputs))
        
        def create_big_image(tensor, single_elem_shape):
            shape = tensor.shape
            tensor = torch.cat([
                t_s.view(*single_elem_shape)
                for t_s in tensor
            ], dim=-1)
            return tensor
        # (batch_size, NUM_CLASSES, H, W) | (batch_size, H, W)
        pred_tensor, target_tensor = collect_prediction_and_target(outputs, self)
        pred_as_mask = take_pred_masks(pred_tensor) # (batch_size, H, W)
        pred_big_tensor = create_big_image(
            pred_tensor, 
            single_elem_shape=[1, NUM_CLASSES, DATA_SHAPE[0], DATA_SHAPE[1]]
        )
        target_big_tensor = create_big_image(
            target_tensor, 
            single_elem_shape=[1, DATA_SHAPE[0], DATA_SHAPE[1]]
        )

        target_one_hotted_tensor = torch.nn.functional.one_hot(
            target_big_tensor, NUM_CLASSES # Num classes
        )
        # (N, H, W, C) --> (N, C, H, W)
        target_one_hotted_tensor = target_one_hotted_tensor.permute(0, -1, 1, 2)
        dice_loss_val = dice_loss(
            pred_big_tensor, target_one_hotted_tensor, 
            dim=[0, 2, 3], use_softmax=True, softmax_dim=1,
        )
        metric = calculate_iou(pred_big_tensor, target_big_tensor)
        
        if self.enable_image_logging:
            for batch_indx, (target_s, pred_s) in enumerate(zip(target_tensor, pred_as_mask)):
                target_s = target_s.squeeze()
                pred_s = pred_s.squeeze()
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
                sns.heatmap(pred_s.cpu().detach().numpy(), ax=ax1, vmin=0, vmax=NUM_CLASSES)
                sns.heatmap(target_s.cpu().detach().numpy(), ax=ax2, vmin=0, vmax=NUM_CLASSES)
                fig.savefig(f'test_unknown_temp_fig_{GPU_ID}.png')
                plt.close(fig)

                if self.experiment is not None and self.current_epoch != 0:
                    self.experiment.log_image(
                        f'test_unknown_temp_fig_{GPU_ID}.png', 
                        name=f'unknown_{batch_indx}', 
                        overwrite=False, step=self.global_step
                    )

        if self.experiment is not None and self.current_epoch != 0:
            self.experiment.log_confusion_matrix(
                target_tensor.cpu().detach().numpy().reshape(-1), 
                torch.stack(
                    [elem.cpu() for elem in pred_as_mask], 
                    dim=0
                ).cpu().detach().numpy().reshape(-1)
            )

        mean_dice_loss_per_class_dict = {
            f"test_mean_dice_loss_per_class_{i}": d_l.float()
            for i, d_l in enumerate(dice_loss_val)
        }
        mean_dice_loss_dict = {
            f"test_mean_dice_loss": dice_loss_val.mean().float()
        }
        mean_iou_class_dict = {
            f"test_mean_iou_class_{i}": torch.tensor(iou, dtype=torch.float)
            for i, iou in enumerate(metric.mean(axis=0))
        }
        mean_iou_dict = {
            "test_mean_iou": float(metric.mean()),
        }
        
        # Log will save stuff in comet
        self.log_dict(mean_iou_dict)
        
        if self.experiment is not None and self.current_epoch != 0:
        
            self.experiment.log_metrics(
                mean_dice_loss_per_class_dict,
                epoch=self.current_epoch
            )

            self.experiment.log_metrics(
                mean_dice_loss_dict,
                epoch=self.current_epoch
            )

            self.experiment.log_metrics(
                mean_iou_class_dict,
                epoch=self.current_epoch
            )

            self.experiment.log_metrics(
                mean_iou_dict,
                epoch=self.current_epoch
            )
        else:
            print(mean_dice_loss_per_class_dict)
            print(mean_dice_loss_dict)
            print(mean_iou_class_dict)
            print(mean_iou_dict)
            print('---------------------------------')


In [ ]:

@nb.njit
def cut_into_parts(
        image: np.ndarray, mask: np.ndarray, h_parts: int, 
        w_parts: int, h_win: int, w_win: int):
    image_parts_list = []
    mask_parts_list = []

    for h_i in range(h_parts):
        for w_i in range(w_parts):
            img_part = image[:, 
                h_i * h_win: (h_i+1) * h_win, 
                w_i * w_win: (w_i+1) * w_win
            ]
            mask_part = mask[
                h_i * h_win: (h_i+1) * h_win, 
                w_i * w_win: (w_i+1) * w_win
            ]

            image_parts_list.append(img_part)
            mask_parts_list.append(mask_part)
    return image_parts_list, mask_parts_list



class DatasetCreator:

    def __init__(
            self, 
            data_path: str,
            preprocessing: Optional[Union[DataPreprocessor, Callable]] = BaseDataPreprocessor(),
            indices = None,
            cut_window=(8, 8),
            map_mask_to_class=False,
            create_shared_memory=False,
            shuffle_then_prepared=False,
            path_old2new: str = None):
        self.dataset = HsiDataset(data_path)
        self.preprocessing = preprocessing
        self.cut_window = cut_window
        self.map_mask_to_class = map_mask_to_class
        self.create_shared_memory = create_shared_memory
        
        if path_old2new:
            self.old2new_mapper = DatasetRemapper(np.load(path_old2new))
        else:
            self.old2new_mapper = None

        self.images = []
        self.masks = []
        self._shm_imgs = None
        self._shm_masks = None

        for idx, data_point in tqdm(enumerate(self.dataset.data_iterator(opened=True, shuffle=False))):
            if indices is not None and idx not in indices:
                continue
            image, mask = data_point.hsi, data_point.mask

            if self.old2new_mapper:
                _, mask = self.old2new_mapper(None, mask)

            if cut_window is not None:
                image_parts, mask_parts = self._cut_with_window(image, mask, cut_window)
                self.images += image_parts
                self.masks += mask_parts
            else:
                self.images.append(image)
                self.masks.append(mask)
        print("Preprocess data...")
        if self.preprocessing is not None:
            self.images, self.masks = self.preprocessing(
                self.images, self.masks, map_mask_to_class=map_mask_to_class
            )

        if shuffle_then_prepared:
            self.images, self.masks = shuffle(self.images, self.masks)

        # Create shared memory
        if create_shared_memory:
            print('Create shared memory...')
            # First - map images and masks into np
            self.images = np.asarray(self.images, dtype=np.float32)
            self.masks = np.asarray(self.masks, dtype=np.int64)
            # Imgs
            shm_imgs = shared_memory.SharedMemory(create=True, size=self.images.nbytes)
            shm_imgs_arr = np.ndarray(self.images.shape, dtype=self.images.dtype, buffer=shm_imgs.buf)
            shm_imgs_arr[:] = self.images[:]
            self.images = shm_imgs_arr # Do not keep dublicate 
            self.data_shm_imgs = ShmData(
                shm_name=shm_imgs.name, shape=self.images.shape, 
                dtype=self.images.dtype
            )
            self._shm_imgs = shm_imgs
            # Masks
            shm_masks = shared_memory.SharedMemory(create=True, size=self.masks.nbytes)
            shm_masks_arr = np.ndarray(self.masks.shape, dtype=self.masks.dtype, buffer=shm_masks.buf)
            shm_masks_arr[:] = self.masks[:]
            self.masks = shm_masks_arr # Do not keep dublicate 
            self.data_shm_masks = ShmData(
                shm_name=shm_masks.name, shape=self.masks.shape,
                dtype=self.masks.dtype
            )
            self._shm_masks = shm_masks
            print("Shared memory are created for imgs and masks!")

    def close_shm(self):
        if self.create_shared_memory:
            # Make sure there is no reference data to images/masks
            del self.images
            del self.masks
            self.images = []
            self.masks = []
            # Close and unlink
            if self._shm_masks is not None:
                self._shm_masks.close()
                self._shm_masks.unlink()
                self._shm_masks = None

            if self._shm_imgs is not None:
                self._shm_imgs.close()
                self._shm_imgs.unlink()
                self._shm_imgs = None
            print("Shared memory for masks and images are success cleared!")
                    
    
    def _cut_with_window(self, image, mask, cut_window):
        assert len(cut_window) == 2
        h_win, w_win = cut_window
        _, h, w = image.shape
        h_parts = h // h_win
        w_parts = w // w_win
        if h % h_win != 0:
            print(f"{h % h_win} pixels will be dropped by h axis. Input shape={image.shape}")

        if w % w_win != 0:
            print(f"{w % w_win} pixels will be dropped by w axis. Input shape={image.shape}")
        return cut_into_parts(
            image=image, mask=mask, h_parts=h_parts, w_parts=w_parts,
            h_win=h_win, w_win=w_win
        )

In [ ]:

def pca_transformation(x, pca_mean, pca_components, pca_explained_variance):
    if len(x.shape) == 3:
        x_t = x.reshape((x.shape[0], -1)) # (C, H, W) -> (C, H * W)
        x_t = np.transpose(x_t, (1, 0)) # (C, H * W) -> (H * W, C)
        x_t = x_t - pca_mean
        x_t = np.dot(x_t, pca_components.T) / np.sqrt(pca_explained_variance)
        return x_t.reshape((x.shape[1], x.shape[2], pca_components.shape[0])).astype(np.float32, copy=False) # (H, W, N)
    elif len(x.shape) == 4:
        # x - (N, C, H, W)
        x_t = np.transpose(x, (0, 2, 3, 1)) # (N, C, H, W) -> (N, H, W, C)
        x_t = x_t - pca_mean
        x_t = np.dot(x_t, pca_components.T) / np.sqrt(pca_explained_variance)
        x_t = np.transpose(x_t, (0, -1, 1, 2)) # (N, H, W, C) -> (N, C, H, W)
        return x_t.astype(np.float32, copy=False)
    else:
        raise ValueError(f"Unknown shape={x.shape}, must be of len 3 or 4.")

def standartization(img, mean, std):
    img -= mean
    img /= std
    return img

def standartization_pool(mean, std):
    # X shape - (N, C, H, W)
    # from shape (comp,) -> (1, comp, 1, 1)
    mean = np.expand_dims(np.expand_dims(np.array(mean, dtype=np.float32), axis=-1), axis=-1)
    std = np.expand_dims(np.expand_dims(np.array(std, dtype=np.float32), axis=-1), axis=-1)
    
    return lambda x: standartization(x, mean=mean, std=std)


def test_augmentation(image, mask, **kwargs):
    image = torch.from_numpy(image)
    #image = (image - image.min()) / (image.max() - image.min())
    
    mask = torch.from_numpy(mask)
    mask = torch.squeeze(mask, 0)
    return image, mask


In [ ]:
kfold = 'kfold0'  # ['kfold0', 'kfold1', 'kfold2']
cur_exp = ATT_EXP  #[ATT_EXP, WO_ATT_EXP, SW_ATT_EXP, SW_WO_ATT_EXP]
run = 'R1' #['R1', 'R2', 'R3', 'R4']
type_indices = 'test' # ['val', 'test']

In [ ]:
net_name = f'{cur_exp} {kfold} {run}'
if kfold == 'kfold1':
    num_classes = 7
else:
    num_classes = 8

if cur_exp in [SW_ATT_EXP, SW_WO_ATT_EXP]:
    PREFIX_INFO_PATH = f'/home/rustam/hyperspecter_segmentation/danil_cave/kfolds_data_with_other/swapped/{kfold}/data'
else:
    PREFIX_INFO_PATH = f'/home/rustam/hyperspecter_segmentation/danil_cave/kfolds_data_with_other/{kfold}/data'
pca_explained_variance = np.load(f'{PREFIX_INFO_PATH}/{kfold}_PcaExplainedVariance_.npy')
pca_mean = np.load(f'{PREFIX_INFO_PATH}/{kfold}_PcaMean.npy')
pca_components = np.load(f'{PREFIX_INFO_PATH}/{kfold}_PcaComponents.npy')
DATA_STANDARTIZATION_PARAMS_PATH = f'{PREFIX_INFO_PATH}/data_standartization_params_{kfold}.json'
with open(DATA_STANDARTIZATION_PARAMS_PATH, 'r') as f:
    DATA_STANDARTIZATION_PARAMS = json.load(f)

pca_transformation_exp = lambda x: pca_transformation(x, pca_mean, pca_components, pca_explained_variance)

def preprocessing(imgs, masks, map_mask_to_class=False, split_size=256): 
    mean, std = (
        DATA_STANDARTIZATION_PARAMS.get('means'), 
        DATA_STANDARTIZATION_PARAMS.get('stds')
    )
    assert mean is not None and std is not None
    print('Create np array of imgs and masks...')
    imgs_np = np.asarray(imgs, dtype=np.float32) # (N, 237, 1, 1)
    masks_np = np.asarray(masks, dtype=np.int64) # (N, 1, 1, 3)
    print("Split imgs dataset...")
    imgs_split_np = np.array_split(imgs_np, split_size) # (split_size, Ns, 237, 1, 1)
    print('Start preprocess images...')
    # Wo PCA
    # _images = [np.transpose(image, (1, 2, 0)) for image in imgs]
    # W Pca
    with Pool(18) as p:
        _images = list(tqdm(p.imap(
                pca_transformation_exp, 
                imgs_split_np,
                #chunksize=1
            ), total=len(imgs_split_np))
        )
        _images = list(tqdm(p.imap(
            standartization_pool(mean=mean, std=std), 
            _images,
            #chunksize=1
            ), total=len(imgs_split_np))
        )
    _images = list(np.concatenate(_images, axis=0)) # (split_size, Ns, 237, 1, 1) -> (split_size * Ns, 237, 1, 1)
    print("Preprocess masks...")
    _masks = list(np.transpose(masks_np[..., 0:1], (0, -1, 1, 2)))
    print("Finish preprocess!")
    return _images, _masks

test_indices = np.load(f'{PREFIX_INFO_PATH}/{kfold}_indx_unknown_test.npy')
val_indices = np.load(f'{PREFIX_INFO_PATH}/{kfold}_indx_test.npy')
train_indices = np.load(f'{PREFIX_INFO_PATH}/{kfold}_indx_train.npy')
path_old2new = f'{PREFIX_INFO_PATH}/index2class.npy'

old2new_mapper = DatasetRemapper(np.load(path_old2new))

if cur_exp in [SW_ATT_EXP, SW_WO_ATT_EXP]:
    w_sorted = sorted(glob.glob(f'swapped_other/pytorch_li_logs/{exp_name2id.get(net_name)}/*'), 
           key=lambda x: -float(x.split('/')[-1].split('-')[-1][9:13])
    )
else:
    w_sorted = sorted(glob.glob(f'original_other/pytorch_li_logs/{exp_name2id.get(net_name)}/*'), 
           key=lambda x: -float(x.split('/')[-1].split('-')[-1][9:13])
    )


pick_best_one = w_sorted[0]
if cur_exp in [ATT_EXP, SW_ATT_EXP]:
    net = MySuperAttentionNetLittleInput(INPUT_DIM, num_classes) 
else:
    net = MySuperNetLittleInput(INPUT_DIM, num_classes) 

model = NnModel.load_from_checkpoint(
    pick_best_one,
    loss=nn.CrossEntropyLoss(), model=net
)
net = model.model
net.eval()
net.to(device=device)

In [ ]:
if type_indices == 'val':
    indices = val_indices
else:
    indices = test_indices

dataset_creator_val = DatasetCreator(
    PATH_DATA, preprocessing=preprocessing, 
    indices=indices, cut_window=DATA_SHAPE,
    create_shared_memory=False
)
dataset_val = HsiDataloaderCutter(
    images=dataset_creator_val.images, masks=dataset_creator_val.masks,
    augmentation=test_augmentation,
    shuffle_data=False, cut_window=DATA_SHAPE,
)
val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=1)

In [ ]:
data_iter = iter(val_loader)

In [ ]:
img_s, mask_s = next(data_iter)

In [ ]:
_, remapped_mask_np = old2new_mapper(None, mask_s.numpy())
img_s = img_s.to(device=device)
pred = net(img_s)
pred = nn.functional.softmax(pred, dim=1)
pred = pred.cpu().detach().numpy()
pred = np.transpose(pred, [0, 2, 3, 1])
pred = np.argmax(pred, axis=-1)

mask_s = torch.squeeze(mask_s).cpu().detach().numpy()

In [ ]:
def get_attention_pac_cam(net, x):
    x = net.backbone(x)
    _, attention_pam = net.pam_module(x, True)
    _, attention_cam = net.cam_module(x, True)
    return attention_pam, attention_cam

In [ ]:
def get_attention_result_pac_cam(net, x):
    x = net.backbone(x)
    attention_pam_result = net.pam_module(x)
    attention_cam_result = net.cam_module(x)
    return attention_pam_result, attention_cam_result

In [ ]:
in_data_x, val_data = next(data_iter)

In [ ]:
# Make predictions
with torch.no_grad():
    data = in_data_x.to(device=device)
    preds = net(data).cpu().detach() # (divided_batch, num_classes)
    preds_attention_pam, preds_attention_cam  = get_attention_pac_cam(net, data)
    preds_attention_pam_result, preds_attention_cam_result  = get_attention_result_pac_cam(net, data)
    preds_attention_pam = preds_attention_pam.cpu().detach()
    preds_attention_cam = preds_attention_cam.cpu().detach()
    preds_attention_pam_result = preds_attention_pam_result.cpu().detach()
    preds_attention_cam_result = preds_attention_cam_result.cpu().detach()

pred_attention_pam_view = preds_attention_pam.view(1, 32, 32, 32, 32)
pred_attention_cam_view = preds_attention_cam.view(1, 128, 128)
pred_attention_pam_view_result = preds_attention_pam_result.view(1, 128, 32, 32)
pred_attention_cam_view_result = preds_attention_cam_result.view(1, 128, 32, 32)
target = val_data.clone()
in_data_x_tensor = in_data_x.clone()
pred_mask = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)

In [ ]:
fig = plt.figure(figsize=(12, 12))

fig.add_subplot(1, 2, 1)
sns.heatmap(pred_mask[0], vmin=0, vmax=NUM_CLASSES)

fig.add_subplot(1, 2, 2)
sns.heatmap(target[0], vmin=0, vmax=NUM_CLASSES)


In [ ]:
indx_b = 0

In [ ]:
pred_argmax = torch.argmax(torch.nn.functional.softmax(preds[indx_b], dim=0), dim=0)
sns.heatmap(pred_argmax, vmin=0, vmax=NUM_CLASSES)

In [ ]:
sns.heatmap(target[indx_b], vmin=0, vmax=NUM_CLASSES)

In [ ]:
x, y = 1, 1
plt.plot(x,y,'bo') 
sns.heatmap(pred_attention_pam_view[indx_b][x, y])

In [ ]:
orig_size = 128
att_size = 32


attention_pam = torch.clone(pred_attention_pam_view[indx_b])

count_x = 4
count_y = 4

size_x = att_size // count_x
size_y = att_size // count_y
y_i, x_i = 0, 0
mid_x, mid_y = size_x // 2, size_y // 2

fig = plt.figure(figsize=(16, 16))

for i in range(1, count_x * count_y + 1):
    fig.add_subplot(count_x, count_y, i)
    if x_i == count_x:
        x_i = 0
        y_i += 1
    x, y = mid_x + x_i * size_x, mid_y + y_i * size_y
    att_s = attention_pam[y, x].unsqueeze(dim=0).unsqueeze(dim=0)
    att_s = torch.nn.functional.interpolate(att_s, (128, 128)).squeeze()
    sns.heatmap(att_s)
    
    x *= (orig_size // att_size)
    y *= (orig_size // att_size)
    plt.plot(x,y,'bo') 
    
    x_i += 1

In [ ]:
indx_show = 0

In [ ]:
fig = plt.figure(figsize=(12, 12))

fig.add_subplot(3, 2, 1,)
ax = sns.heatmap((-1)*pred_attention_cam_view_result[indx_show, 0])
ax.set(xlabel='cam')

fig.add_subplot(3, 2, 2)
ax = sns.heatmap((-1)*pred_attention_pam_view_result[indx_show, 0])
ax.set(xlabel='pam')

fig.add_subplot(3, 2, 3)
ax = sns.heatmap(in_data_x_tensor[indx_show, 0])
ax.set(xlabel='input')

fig.add_subplot(3, 2, 4)
ax = sns.heatmap(target[indx_show], vmin=0, vmax=NUM_CLASSES)
ax.set(xlabel='mask')

fig.add_subplot(3, 2, 5)
pred_argmax = torch.argmax(torch.nn.functional.softmax(preds[indx_show], dim=0), dim=0)
ax = sns.heatmap(pred_argmax, vmin=0, vmax=NUM_CLASSES)
_ = ax.set(xlabel='pred')


In [ ]:
fig = plt.figure(figsize=(12, 12))

fig.add_subplot(3, 2, 1,)
ax = sns.heatmap(pred_attention_cam_view_result[indx_show].mean(axis=0))
ax.set(xlabel='cam')

fig.add_subplot(3, 2, 2)
ax = sns.heatmap(pred_attention_pam_view_result[indx_show].mean(axis=0))
ax.set(xlabel='pam')

fig.add_subplot(3, 2, 3)
ax = sns.heatmap(in_data_x_tensor[indx_show, 0])
ax.set(xlabel='input')

fig.add_subplot(3, 2, 4)
ax = sns.heatmap(target[indx_show], vmin=0, vmax=NUM_CLASSES)
ax.set(xlabel='mask')

fig.add_subplot(3, 2, 5)
pred_argmax = torch.argmax(torch.nn.functional.softmax(preds[indx_show], dim=0), dim=0)
ax = sns.heatmap(pred_argmax, vmin=0, vmax=NUM_CLASSES)
_ = ax.set(xlabel='pred')


In [ ]:
sns.heatmap(in_data_x_tensor[indx_b][0])

In [ ]:
fig = plt.figure(figsize=(16, 16))

for i in range(1, 17):
    fig.add_subplot(count_x, count_y, i)
    sns.heatmap(in_data_x_tensor[indx_b][i])

In [ ]:
s_data_x = in_data_x_tensor[indx_b]
in_data_x_tensor_normed = (s_data_x - s_data_x.min()) / (s_data_x.max() - s_data_x.min())

In [ ]:
sns.heatmap(in_data_x_tensor_normed[0])

In [ ]:
fig = plt.figure(figsize=(16, 16))

for i in range(1, 17):
    fig.add_subplot(count_x, count_y, i)
    sns.heatmap(in_data_x_tensor_normed[i])

In [ ]:
sns.heatmap(torch.sqrt(torch.sum(torch.square(in_data_x_tensor[indx_b]), dim=0)))

In [ ]:
sns.heatmap(torch.sum(in_data_x_tensor[indx_b], dim=0))

In [ ]:
plt.plot(x,y,'bo') 
sns.heatmap(pred_attention_cam[indx_b])

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(
    np.asarray(target_list).reshape(-1),
    np.asarray(preds_list).reshape(-1), 
    average='macro'
)

In [ ]:
f1_score(
    np.asarray(target_list).reshape(-1),
    np.asarray(preds_list).reshape(-1), 
    average='weighted'
)